# Relazione Finale - Gruppo Dig Data
Componenti Gruppo - Alexandru Pavel, Simone Garzarella

# Indice
TODO: Mettere i numeri sezioni solo qui per chiarezza ulteriore (anche se non serve)
- [Introduzione](#introduction)
    *  [Descrizione Problema](#problem-description)
    *  [Specifiche Software](#sw-specs)
    *  [Specifiche Software](#hw-specs)
- [Analisi Dataset](#data-analysis)
    * [Historical Stock Prices](#hsp)
    * [Historical Stocks](#hs)
- [Job 1](#job1)
    * [MapReduce](#mapreduce1)
    * [Hive](#hive1)
    * [Spark](#spark1)
- [Job 2](#job2)
    * [MapReduce](#mapreduce2)
    * [Hive](#hive2)
    * [Spark](#spark2)
- [Job 3](#job3)
    * [MapReduce](#mapreduce3)
    * [Hive](#hive3)
    * [Spark](#spark3)
- [Grafici Finali](#plots)
    * [Tipo1](#plot1)
    * [Tipo2](#plot2)
    * [Tipo3](#plot3)
- [Conclusioni](#conclusions)

# Introduzione <a name="introduction"></a>
Il dataset "Daily Historical Stock Prices" contiene l'andamento delle azioni sulla borsa di New York (NYSE e NASDAQ) dal 1970 al 2018. 

Due file CSV compongono il dataset:
- historical_stock_prices.csv
- historical_stocks.csv

Il primo contiene i valori dei prezzi e volumi che variano nel tempo per ogni ticker. Il secondo i dati relativi ad ogni ticker, come il settore e l'exchange in cui è quotato.

## Descrizione Problema <a name="problem-description"></a>
Dopo una fase iniziale di analisi e processamento di dati si vogliono eseguire 3 job (descritti nel dettaglio più avanti) con le diverse tecnologie affrontate nel corso (Hadoop, Hive e Apache Spark).

## Specifiche Software <a name="sw-specs"></a>
Serve? Python, Hadoop, Hive, Spark con versioni

## Specifiche Hardware <a name="hw-specs"></a>
I test sono stati eseguiti in locale e su cluster con macchine con queste caratteristiche.
### Locale
Ubuntu 20.04, CPU i5 2.5GHZ, 8GB Ram e 256GB SSD
### Cluster
Amazon Web Services EMR su Cluster con 1 Master Node e 5 Data Node. Tutte le istanze sono m5.xlarge con 16GB RAM, 4 vCPU e 50GB SSD.

# Analisi Dataset <a name="data-analysis"></a>

Di seguito vengono analizzati i due file del dataset per individuare eventuali preprocessamenti da effettuare. Inoltre viene anche descritto il processo per creare dataset più piccoli o grandi (con sampling) per effettuare i successivi test.

In [4]:
import pandas as pd

## Historical Stock Prices <a name="hsp"></a>

I campi di questo dataset sono:

- `ticker`: simbolo univoco dell’azione (https://en.wikipedia.org/wiki/Ticker_symbol)
- `open`: prezzo di apertura
- `close`: prezzo di chiusura
- `adj_close`: prezzo di chiusura “modificato”
- `lowThe`: prezzo minimo
- `highThe`: prezzo massimo
- `volume`: numero di transazioni
- `date`: data nel formato aaaa-mm-gg

In [10]:
stock_prices = pd.read_csv('dataset/historical_stock_prices.csv')

In [11]:
stock_prices

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14
...,...,...,...,...,...,...,...,...
20973884,NZF,14.60,14.59,14.590000,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.580000,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.590000,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.570000,14.57,14.64,135600,2018-08-23


Ci sono ~21milioni di record per questo file

In [12]:
stock_prices.isna().sum()

ticker       0
open         0
close        0
adj_close    0
low          0
high         0
volume       0
date         0
dtype: int64

Non sono presenti valori nulli per nessuna delle colonne

In [13]:
stock_prices.nunique()

ticker          5685
open          807688
close         835181
adj_close    9235753
low           815237
high          821044
volume        385849
date           12274
dtype: int64

In totale ci sono 5685 `ticker` univoci nel dataset

In [14]:
stock_prices[stock_prices.duplicated(subset=['ticker','date'])].shape

(0, 8)

Non ci sono record distinti con valori duplicati di (ticker, data)

### Creazione di dataset di varie dimensioni

Sono stati generati dataset di dimensioni (approssimativamente) di 256/512/1024MB e ~4GB, oltre al dataset originale che ha dimensioni ~2GB.

I file generati (con relativa dimensione precisa) hanno nome historical_stock_prices[size].csv

- historical_stock_prices`256`.csv &ensp;&ensp;(239.75MB)
- historical_stock_prices`512`.csv &ensp;&ensp;(479.51MB)
- historical_stock_prices`1024`.csv &ensp;(959.03MB)
- historical_stock_prices.csv &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(1909.97MB)
- historical_stock_prices`4096`.csv &ensp;(3835.92MB)

La scelta dei record da includere è effettuata con un sampling randomico (con un seed preimpostato, per la ripetibiltà)

```python
def sample_all_sizes(historical_stock_prices_df):
    for size in [0.125, 0.25, 0.5, 2]:
        sample_n_rows = dataset_row_count * size
        sampled_df = dataset.sample(sample_n_rows)
        filename = 'dataset/historical_stock_prices[SIZE].csv')
        sampled_df.to_csv(filename)

```

## Historical Stocks <a name="hs"></a>

Il dataset con le informazioni sui ticker è così strutturato:

- `ticker`: simbolo dell’azione
- `exchange`: NYSE o NASDAQ
- `name`: nome dell’azienda
- `sector`: settore dell’azienda
- `industry`: industria di riferimento per l’azienda

In [15]:
stocks = pd.read_csv("dataset/historical_stocks.csv")

In [16]:
stocks

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS
...,...,...,...,...,...
6455,ZOES,NYSE,"ZOE&#39;S KITCHEN, INC.",CONSUMER SERVICES,RESTAURANTS
6456,ZTS,NYSE,ZOETIS INC.,HEALTH CARE,MAJOR PHARMACEUTICALS
6457,ZTO,NYSE,ZTO EXPRESS (CAYMAN) INC.,TRANSPORTATION,TRUCKING FREIGHT/COURIER SERVICES
6458,ZUO,NYSE,"ZUORA, INC.",TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE


In [18]:
stocks.nunique()

ticker      6460
exchange       2
name        5462
sector        13
industry     136
dtype: int64

Sono presenti 6460 `ticker` univoci, come il numero di righe del dataset. Il ticker può essere considerato una chiave di questo dataset, il nome dell'azienda `name` invece no, ha delle ripetizioni.

In [17]:
stocks[stocks.duplicated(subset=['name'])].shape

(998, 5)

In particolare sono presenti 998 nomi di azienda duplicati. Nel resto del progetto non si considererà questo campo per identificare record (in particolare per il job3).

In [20]:
stocks['sector'].unique()

array(['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES',
       'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE',
       'CONSUMER DURABLES', nan, 'ENERGY', 'MISCELLANEOUS', 'SECTOR',
       'TRANSPORTATION', 'CONSUMER NON-DURABLES'], dtype=object)

Visualizzando i possibili valori di `sector` si può notare la presenza di un valore nullo.

In [19]:
stocks.isna().sum()

ticker         0
exchange       0
name           0
sector      1440
industry    1440
dtype: int64

Il campo `sector` presenta 1440 valori nulli, che vengono eliminati durante il preprocessing di questo dataset.

In [21]:
stocks_clean = stocks.loc[(stocks['sector'].notna())]

In [23]:
stocks_clean.shape

(5020, 5)

Il dataset pulito dai valori nulli del campo `sector` ha 5020 record. Verrà salvato come `historical_stocks_clean.csv`

```python
stocks_clean.to_csv('dataset/historical_stocks_clean.csv')
```

# Job 1 <a name="job1"></a>

Un job che sia in grado di generare un report contenente, per ciascuna azione: 
* (a) la data della prima quotazione,
* (b) la data dell’ultima quotazione, 
* (c) la variazione percentuale della quotazione (differenza percentuale tra il primo e l’ultimo prezzo di chiusura presente nell’archivio),
* (d) il prezzo massimo e quello minimo,
* (e) (facoltativo) il massimo numero di giorni consecutivi in cui l’azione è cresciuta(chiusura maggiore dell’apertura)con indicazione dell’anno in cui questo è avvenuto.

Il report deve essere ordinato per valori decrescenti del punto b. 

## MapReduce <a name="mapreduce1"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="hive1"></a>

Per eseguire questo job sono state create prima due tabelle esterne:

* Una che, per ogni ticker, etrae il prezzo di chiusura alla prima data disponibile per quel ticker nel database:


`create table ticker_to_minDate as
 select d.ticker as min_ticker, d.close_price as min_close_price
 from historical_stock_prices(size) d
 join (select ticker as min_ticker, min(price_date) as min_price_date
 from historical_stock_prices(size) group by ticker) min_table
 on (d.ticker = min_table.min_ticker and d.price_date <= min_table.min_price_date);`
 
* L'altra che, per ogni ticker, etrae il prezzo di chiusura allultima data disponibile per quel ticker nel database:

`create table ticker_to_maxDate as
 select d.ticker as max_ticker, d.close_price as max_close_price
 from historical_stock_prices(size) d
 join (select ticker as max_ticker, max(price_date) as max_price_date
 from historical_stock_prices(size) group by ticker) as max_table
 on (d.ticker = max_table.max_ticker and d.price_date >= max_table.max_price_date);`


* Successivamente esse sono state utilizzate per la query finale, in cui si estrae, per ogni ticker, la data della prima quotazione, la data dell’ultima quotazione, la variazione percentuale della quotazione, il prezzo massimo e quello minimo.

`CREATE TABLE job1_hive ROW FORMAT DELIMITED
 FIELDS TERMINATED BY '\t'
 LINES TERMINATED BY '\n' as
 select ticker, min(price_date) as first_price_date, max(price_date) as last_price_date,
 max(((max_table.max_close_price - min_table.min_close_price) / min_table.min_close_price) * 100) as variation,
 max(high) as max_price, min(low) as min_price
 from historical_stock_prices(size) d
 join ticker_to_maxDate max_table on d.ticker = max_table.max_ticker
 join ticker_to_minDate min_table on d.ticker = min_table.min_ticker
 group by ticker
 order by ticker, last_price_date desc;`


## Spark <a name="spark1"></a>
This is a sub paragraph, formatted in heading 3 style

# Job 2 <a name="job2"></a>

Un  job  che  sia in  grado  di  generare un report contenente, per  ciascun settore  e  per  ciascun anno del  periodo  2009-2018: 

* (a) la  variazione percentuale della quotazione del settore nell’anno, 
* (b) l’azione del settore che ha avuto il maggior incremento percentuale nell’anno (con indicazione dell’incremento),
* (c) l’azione del settore che ha avuto il maggior  volume  di  transazioni nell’anno(con  indicazione  del  volume).

Il  report  deve  essere  ordinato  per  nome  del settore. 


## MapReduce <a name="mapreduce2"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="hive2"></a>

Per questo job sono state create diverse tabelle esterne, per chiarezza divise a seconda del task per cui esse hanno un'utilità:

UTILI PER IL TASK A

* Per ogni settore estrare l'anno dalle date dei prezzi


`create table sector_2_date as
select distinct d2.sector, extract(year from d1.price_date)
from historical_stock_prices(size) as d1 left join historical_stocks_clean as d2 on d1.ticker = d2.ticker
order by d2.sector, `_c1`;
alter table sector_2_date change `_c1` year int;`

* Per ogni settore, anno e ticker estrae la data della prima e dell'ultima quotazione dell'anno.


`create table sector_ticker_min_max as
select d2.sector, sd.year, d1.ticker, min(d1.price_date) as first_date, max(d1.price_date) as last_date
from historical_stock_prices(size) as d1
left join historical_stocks_clean as d2 on d1.ticker = d2.ticker
left join sector_2_date as sd on d2.sector = sd.sector and sd.year = extract(year from d1.price_date)
where sd.year >=2009 and sd.year <= 2018
group by d2.sector, sd.year, d1.ticker
order by sector, year, d1.ticker;`

* Per ogni settore e anno calcola la somma di tutte e quotazioni nella prima data dell'anno per quel settore.

`create table sector_to_min_quot as
select d2.sector, sm.year, sum(d1.close_price) as first_quot
from historical_stock_prices(size) as d1
left join historical_stocks_clean as d2 on d1.ticker = d2.ticker
join sector_ticker_min_max as sm on d2.sector = sm.sector and sm.year = extract(year from d1.price_date) and d1.ticker = sm.ticker
where d1.price_date = sm.first_date
group by d2.sector, sm.year
order by d2.sector, sm.year;`


* Per ogni settore e anno calcola la somma di tutte e quotazioni nell'ultima data dell'anno per quel settore.

`create table sector_to_max_quot as
select d2.sector, sm.year, sum(d1.close_price) as last_quot
from historical_stock_prices(size) as d1
left join historical_stocks_clean as d2 on d1.ticker = d2.ticker
join sector_ticker_min_max as sm on d2.sector = sm.sector and sm.year = extract(year from d1.price_date) and d1.ticker = sm.ticker
where d1.price_date = sm.last_date and  d2.sector != "N/A"
group by d2.sector, sm.year
order by d2.sector, sm.year;`

UTILI PER IL TASK B

* Per ogni settore e anno estrae il ticker con la sua prima quotazione per quel settore e in quell'anno.


`create table sector_year_to_tickerFirstQuotation as
select d2.sector, sm.year, d1.ticker, close_price as first_quotation
from historical_stock_prices(size) as d1
left join historical_stocks_clean as d2 on d1.ticker = d2.ticker
left join sector_ticker_min_max as sm on d2.sector = sm.sector and d1.ticker = sm.ticker
where d1.price_date = sm.first_date
order by d2.sector, sm.year;`


* Per ogni settore e anno estrae il ticker con la sua ultima quotazione per quel settore e in quell'anno.

`create table sector_year_to_tickerLastQuotation as
select d2.sector, sm.year, d1.ticker, close_price as last_quotation
from historical_stock_prices(size) as d1
left join historical_stocks_clean as d2 on d1.ticker = d2.ticker
left join sector_ticker_min_max as sm on d2.sector = sm.sector and d1.ticker = sm.ticker
where d1.price_date = sm.last_date
order by d2.sector, sm.year;`

* Per ogni settore e anno estrae il ticker con la sua prima e ultima quotazione per quel settore e in quell'anno (Join delle due tabelle precedenti).

`create table sector_year_to_tickerFirstLastQuotation as
select s1.sector, s1.year, s1.ticker, s1.first_quotation, s2.last_quotation
from sector_year_to_tickerFirstQuotation as s1
left join sector_year_to_tickerLastQuotation as s2
on (s1.sector = s2.sector and s1.year = s2.year and s1.ticker = s2.ticker)
order by s1.sector, s1.year;`


* Per ogni settore, anno e ticker calcola la variazione percentuale del ticker in quell'anno per quel settore.

`create table sector_year_to_variation as
select sector, year, ticker, max(((last_quotation - first_quotation)/first_quotation)*100) as variation
from sector_year_to_tickerFirstLastQuotation
group by sector, year, ticker;`


* Per ogni settore e anno calcola la variazione massima avuta in quell'anno e per quel settore.

`create table sector_year_to_maxVariation as
select sector, year, max(variation) as max_variation
from sector_year_to_variation
group by sector, year;`


* Per ogni settore e anno estrae il ticker che ha avuto la variazione percentuale massima in quell'anno e per quel settore, con l'indicazione di tale variazione.

`create table sector_year_to_maxTicker as
select smax.sector, smax.year, sv.ticker, smax.max_variation
from sector_year_to_maxVariation as smax
left join sector_year_to_variation as sv on smax.sector = sv.sector and smax.year = sv.year
where max_variation = variation;`


UTILI PER IL TASK C

* Per ogni settore, anno e ticker calcola la somma dei volumi dei ticker in quell'anno e per quel settore.

`create table sector_year_ticker_to_volumeSum as
select d2.sector, year(d1.price_date) as price_year, d1.ticker, sum(d1.volume) as volume
from historical_stock_prices(size) as d1
join historical_stocks_clean as d2 on d1.ticker = d2.ticker
group by d2.sector, year(d1.price_date), d1.ticker;`

* Per ogni settore e anno estrae la somma di volumi massima in quell'anno e per quel settore.

`create table sector_year_to_maxVolume as
select sector, price_year, max(volume) as maxVolume
from sector_year_ticker_to_volumeSum
group by sector, price_year
order by sector, price_year;`

* Per ogni settore e anno estrae il ticker che ha la somma di volumi massima in quell'anno e per quel settore, con indicazione di tale somma.

`create table sector_year_toMaxVolumeTicker as
select ayt.sector, ayt.price_year, ayt.ticker as v_ticker, ayt.volume
from sector_year_ticker_to_volumeSum as ayt
left join sector_year_to_maxVolume as aym on ayt.sector = aym.sector and ayt.price_year = aym.price_year
where volume = maxVolume;`

QUERY FINALE

* Mette insieme tutte le precedenti tabelle per estrarre, per ogni settore e anno, la  variazione percentuale della quotazione del settore nell’anno, l’azione del settore che ha avuto il maggior incremento percentuale nell’anno (con indicazione dell’incremento), l’azione del settore che ha avuto il maggior  volume  di  transazioni nell’anno(con  indicazione  del  volume).

`create table job2_hive as
select d2.sector, smin.year, min(((smax.last_quot - smin.first_quot)/smin.first_quot)*100) as variation, max(sy.ticker), max(sy.max_variation), min(v_ticker), max(syv.volume)
from historical_stock_prices(size) as d1
left join historical_stocks_clean as d2 on d1.ticker = d2.ticker
left join sector_to_min_quot as smin on d2.sector = smin.sector and smin.year = extract(year from d1.price_date)
left join sector_to_max_quot as smax on d2.sector = smax.sector and smax.year = extract(year from d1.price_date)
left join sector_year_to_maxTicker sy on d2.sector = sy.sector and sy.year = extract(year from d1.price_date)
left join sector_year_toMaxVolumeTicker as syv on d2.sector = syv.sector and syv.price_year = extract(year from d1.price_date)
where smin.year >=2009 and smin.year <= 2018 and smax.year >=2009 and smax.year <= 2018 and d2.sector != "N/A"
group by d2.sector, smin.year
order by d2.sector, smin.year;`



## Spark <a name="spark2"></a>
This is a sub paragraph, formatted in heading 3 style

# Job 3 <a name="job3"></a>

Un job in grado di generare le coppie di aziende che si somigliano (sulla base di una soglia scelta a piacere) in termini di variazione percentuale mensile nell’anno 2017 mostrando l’andamento mensile delle due aziende.

(Es.Soglia=1%, coppie: 1:{Apple, Intel}:GEN: Apple +2%, Intel +2,5%, FEB: Apple +3%, Intel +2,7%, MAR: Apple +0,5%, Intel +1,2%, ...; 2:{Amazon, IBM}: GEN: Amazon +1%, IBM +0,5%, FEB: Amazon +0,7%, IBM +0,5%,MAR: Amazon +1,4%, IBM +0,7%,..)

## MapReduce <a name="mapreduce3"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="hive3"></a>

Per l'ultimo job vengono create diverse tabelle esterne che saranno poi utilizzate per costruire la tabella finale:

* Filtra il database per mostrare solo i dati del 2017

`create table 2017_data as
select ticker, price_date, extract(month from price_date), close_price
from historical_stock_prices(size)
where extract(year from price_date) = 2017
order by ticker, price_date;
alter table 2017_data change `_c2` month int;`

* Per ogni ticker e mese estrae il primo e l'ultimo prezzo di chiusura del ticker in quel mese

`create table ticker_month_to_max_min_date as
select ticker, month, min(price_date) as min_date, max(price_date) as max_date
from 2017_data
group by ticker, month;`

* Per ogni ticker e mese estrae la prima quotazione di quel ticker in quel mese

`create table ticker_to_first_month_quotation as
select d.ticker, d.month, d.close_price
from 2017_data as d
left join ticker_month_to_max_min_date as tm on d.ticker = tm.ticker
where price_date = min_date;`


* Per ogni ticker e mese estrae l'ultima quotazione di quel ticker in quel mese

`create table ticker_to_last_month_quotation as
select d.ticker, d.month, d.close_price
from 2017_data as d
left join ticker_month_to_max_min_date as tm on d.ticker = tm.ticker
where price_date = max_date;`

* Per ogni ticker e mese estrae la prima e l'ultima quotazione del ticker in quel mese (join tra le due tabelle precedenti)

`create table ticker_to_first_last_month_quotation as
select first.ticker, first.month, first.close_price as first_quotation, last.close_price as last_quotation
from ticker_to_first_month_quotation as first
left join ticker_to_last_month_quotation as last
on first.ticker = last.ticker and first.month = last.month
order by ticker, month;`

* Per ogni ticker e mese calcola la variazione percentuale di quel ticker in quel mese

`create table ticker_month_to_variation as
select ticker, month, (((last_quotation - first_quotation)/first_quotation)*100) as variation
from ticker_to_first_last_month_quotation
order by ticker, month;`

* Per ogni coppia di ticker e per ogni mese estrae la variaizone percentuale del primo e del secondo ticker per quel mese 

`create table variations_comparison as
select t1.ticker as ticker_1, t2.ticker as ticker_2, t1.month, cast(t1.variation as decimal(10,2)) as variation_1, cast(t2.variation as decimal(10,2)) as variation_2
from ticker_month_to_variation as t1, ticker_month_to_variation as t2
where t1.ticker > t2.ticker and t1.month = t2.month and (abs(t1.variation - t2.variation) <= 1)
order by ticker_1, ticker_2, t1.month;`

* Crea la tabella dei risultati raggruppando per coppie di ticker e trasformando i valori della colonna "month" in nuove colonne, ognuna per ogni mese, popolate dai rispettivi valori per quel mese.

`create table raw_results as
select ticker_1 as t1, ticker_2 as t2,
max(case when month="1" then "GEN:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as gen,
max(case when month="2" then "FEB:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as feb,
max(case when month="3" then "MAR:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as mar,
max(case when month="4" then "APR:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as apr,
max(case when month="5" then "MAG:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as mag,
max(case when month="6" then "GIU:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as giu,
max(case when month="7" then "LUG:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as lug,
max(case when month="8" then "AGO:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as ago,
max(case when month="9" then "SET:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as sep,
max(case when month="10" then "OTT:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as ott,
max(case when month="11" then "NOV:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as nov,
max(case when month="12" then "DIC:"||" "||"("||variation_1||"%"||", "||variation_2||"%"||")" else "" end) as dic
from variations_comparison
group by ticker_1, ticker_2;`

* Nel mostrare i risutati si filtra la tabella finale affinché mostri soltanto le coppie di ticker che si somigliano in tutti i mesi.

`create table job3_hive as
select * from raw_results
where (gen!="" and feb!="" and mar!="" and apr!="" and mag!="" and giu!="" and lug!="" and ago!=""
and sep!="" and ott!="" and nov!="" and dic!="");`


## Spark <a name="spark3"></a>
This is a sub paragraph, formatted in heading 3 style

# Grafici <a name="plots"></a>
The first paragraph text

## MapReduce <a name="plot1"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="plot2"></a>
This is a sub paragraph, formatted in heading 3 style

## Spark <a name="plot3"></a>
This is a sub paragraph, formatted in heading 3 style

# Conclusioni <a name="conclusions"></a>
The first paragraph text
